In [42]:
import pandas as pd

df = pd.read_csv("../raw_data/en.openfoodfacts.org.products.csv", delimiter='\t')

/home/qtzzz/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0,8,13,19,20,21,22,23,27,28,29,31,32,40,41,42,52,55,64,66) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Nous tout d'abord supprimer les colonnes avec plus de 50% de missing values

In [43]:
print('NaN rate per colum:')
for col in df.columns:
    sum = df[col].isna().sum()
    print(col + ': ' + str(sum/len(df)*100) + '%')
    if sum/len(df)*100 > 50:
        df.drop(columns=col, inplace=True)

NaN rate per colum:
code: 0.0%
url: 0.0%
creator: 0.00019423529080422148%
created_t: 0.0%
created_datetime: 0.0%
last_modified_t: 0.0%
last_modified_datetime: 0.0%
product_name: 4.048688960345894%
abbreviated_product_name: 99.64945385892108%
generic_name: 93.99730401416365%
quantity: 74.77296322446122%
packaging: 84.63054990924356%
packaging_tags: 84.6324922621516%
packaging_text: 99.59793294803526%
brands: 48.86571446052605%
brands_tags: 48.86911357811512%
categories: 54.05796369548179%
categories_tags: 54.05835216606341%
categories_en: 54.05835216606341%
origins: 95.70074751451666%
origins_tags: 95.7070601614678%
origins_en: 95.7070601614678%
manufacturing_places: 93.75266466539573%
manufacturing_places_tags: 93.75552963593509%
labels: 78.67315930498728%
labels_tags: 78.67413048144131%
labels_en: 78.67413048144131%
emb_codes: 93.98676674963751%
emb_codes_tags: 93.98900045548176%
first_packaging_code_geo: 96.40756973775322%
cities: 100.0%
cities_tags: 96.12903633073996%
purchase_place

In [44]:
print(df.columns)

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name', 'brands',
       'brands_tags', 'countries', 'countries_tags', 'countries_en',
       'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_en',
       'image_url', 'image_small_url', 'image_nutrition_url',
       'image_nutrition_small_url', 'energy-kcal_100g', 'energy_100g',
       'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g'],
      dtype='object')


Nous pouvons maintenant supprimer certaines colonnes qui n'apporteront rien pour le clustering:
- ```urlcreator, created_t, created_datetime, states, states_tags, last_modified_datetime, last_modified_t, image_url, image_small_url, image_nutrition_url, image_nutrition_small_url, states, states_en``` &rarr; Variables portant sur l'entrée du produit dans la bdd openfoodfacts
- ```product_name``` &rarr; A étudier lorsque la variable sera encodée pour voir les changements sur le modèle
- ```countries, countries_en``` &rarr; Redondant avec country_tags
- ```nutrition-score-fr_100g``` &rarr; Redondant, le nutriscore est calculé par rapport aux quantités de proteines/lipides/glucides présentent dans le produit
- ```energy_100g``` &rarr; redondant avec ```energy-kcal_100g```
- ```brands et brands_tags``` &rarr; Nom de la marque et nom de la marque sans caractères spéciaux.Un certain nombre de Nan mais une information qui est parfois utile lorsqu'elle est présente (Intuitivement par exemple nous savons qu'un produit de la marque Picard sera très probablement un produit surgelé). A étudier lorsqu'encodée
-```code``` &rarr; Cette variable correspond au code EAN13. Une partie du code correspond au pays et à l'entreprise emmetrice du produit [source](https://www.laintimes.com/)
A réencoder en 3 colonnes différentes, `Pays`, `Entreprise`, `Code Produit` et supprimer le dernier chiffre

In [50]:
columns = ['url', 'creator', 'created_t', 'created_datetime', 'states', 'states_tags', 'last_modified_datetime', 'last_modified_t',
'image_url', 'image_small_url', 'image_nutrition_url', 'image_nutrition_small_url', 'countries','countries_en', 'nutrition-score-fr_100g',
'energy_100g', 'product_name', 'states', 'states_en', 'brands', 'brands_tags']

df.drop(columns=df.columns.intersection(columns), inplace=True)
# to_drop = [col for col in columns if col in df.columns]
# df.drop(columns=to_drop, inplace=True)


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059358 entries, 0 to 2059357
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   code                object 
 1   pnns_groups_1       object 
 2   pnns_groups_2       object 
 3   fat_100g            float64
 4   saturated-fat_100g  float64
 5   carbohydrates_100g  float64
 6   sugars_100g         float64
 7   proteins_100g       float64
 8   salt_100g           float64
 9   sodium_100g         float64
dtypes: float64(7), object(3)
memory usage: 157.1+ MB


Les colonnes restantes:
explication-et-construction-des-codes-barres-suivant-la-norme-ean-13/)
- pnns_groups_1 et pnns_groups_2 &rarr; Ces colonnes correspondent deja 
- fat_100g, saturated-fat_100g, carbohydrates_100g, sugars_100g, proteins_100g, salt_100g, sodium_100g &rarr; Valeurs nutritionelles
- ```energy-kcal_100g``` &rarr; Les fabricants de produits alimentaires calculent cette valeur en additionnant où kcal/g de protéines, lipides et glucides contenus dans leur produit. [source](https://fr.wikipedia.org/wiki/Composition_nutritionnelle_des_aliments) &rarr; à vérifier si cette valeur n'est pas redondante avec les valeurs nnutritionelles
- ```serving_size``` &rarr; La taille d'une portion n'a pas de lien avec la catégorie du produit.